<a href="https://colab.research.google.com/github/imadmlf/GroundingDINO/blob/main/DINO_APP_BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install streamlit -q

In [ ]:
!nvidia-smi

Mon May 27 13:55:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!git clone https://github.com/imadmlf/GroundingDINO.git
%cd {HOME}/GroundingDINO
!git checkout -q 57535c5a79791cb76e36fdb64975271354f10251
!pip install -q -e .

fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/content/GroundingDINO
fatal: reference is not a tree: 57535c5a79791cb76e36fdb64975271354f10251
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-gc2_o7it
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-gc2_o7it
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
!pip install supervision==0.12.0

In [ ]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights
!wget -q https://github.com/imadmlf/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [ ]:
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
%cd {HOME}
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

/content
cuda:0


In [ ]:
GROUNDING_DINO_CONFIG_PATH = '/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
GROUNDING_DINO_CHECKPOINT_PATH = '/content/weights/groundingdino_swint_ogc.pth'

In [ ]:
import sys
sys.path.append("/content/GroundingDINO/groundingdino")

In [ ]:
!pip install Pillow

In [ ]:
# Check the model_checkpoint_path
print(GROUNDING_DINO_CHECKPOINT_PATH)

# Download the checkpoint file again if necessary
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -O {GROUNDING_DINO_CHECKPOINT_PATH}


/content/weights/groundingdino_swint_ogc.pth


In [ ]:
from groundingdino.util.inference import Model

In [ ]:
GD_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)


/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install requests


In [ ]:
# annotation and mask
%%writefile App.py
import streamlit as st
import cv2
import numpy as np
from groundingdino.util.inference import Model
import supervision as sv
import torch
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
import requests



def app0():
    st.title('Image Segmentation Prompt')
    st.write("Welcome to the Image Segmentation and Annotation Application. This guide will help you understand how to use the different features of this application.")

    st.header("Navigation Sidebar")
    st.write("""
        Use the navigation sidebar on the left to switch between different functionalities:
        - **Home**: This page, which provides an overview of the application.
        - **Annotate Images**: Upload images and annotate them by drawing bounding boxes around specified classes.
        - **Mask Images**: Upload images and apply masks to specified objects within the images.
        - **Mask and Annotate Images**: Combine both annotation and masking functionalities.
    """)
    # Displaying an image from GitHub
    url1 = r'https://github.com/imadmlf/GroundingDINO/blob/main/streamlit/1.png?raw=true'
    resp = requests.get(url1, stream=True).raw
    image_array_heart = np.asarray(bytearray(resp.read()), dtype="uint8")
    image_heart = cv2.imdecode(image_array_heart, cv2.IMREAD_COLOR)

    st.image(cv2.cvtColor(image_heart, cv2.COLOR_BGR2RGB), caption='Navigation sidebar image')

    st.header("How to Use the Application")
    st.write("""
        Follow these steps to use the application:

        1. **Select a Functionality**:
            - Use the sidebar to choose between annotating images, masking images, or doing both.

        2. **Upload Images**:
            - Click the "Upload images" button to select and upload images from your computer. You can upload multiple images at once.

        3. **Enter Classes**:
            - In the text area provided, enter the classes you want to detect, separated by commas (e.g., "person, chair").

        4. **Process Images**:
            - Click the "Annotate Images" or "Mask Images" button (depending on the selected functionality) to process the uploaded images.

        5. **View Results**:
            - The original and processed images (annotated or masked) will be displayed side by side for comparison.
    """)

    st.header("Detailed Explanation of Each Functionality")

    st.subheader("Annotate Images")
    st.write("""
        In this section, you can upload images and annotate them with bounding boxes around specified classes. The steps are as follows:
        - Upload the images.
        - Enter the classes you want to detect.
        - Click the "Annotate Images" button to see the annotated images.
    """)
    # Displaying an image from GitHub
    url2 = r'https://github.com/imadmlf/GroundingDINO/blob/main/streamlit/2.PNG?raw=true'
    resp = requests.get(url2, stream=True).raw

    image_array_heart = np.asarray(bytearray(resp.read()), dtype="uint8")
    image_heart = cv2.imdecode(image_array_heart, cv2.IMREAD_COLOR)

    st.image(cv2.cvtColor(image_heart, cv2.COLOR_BGR2RGB), caption='Annotate images')

    st.subheader("Mask Images")
    st.write("""
        In this section, you can upload images and apply masks to specified objects within the images. The steps are as follows:
        - Upload the images.
        - Enter the classes you want to detect.
        - Click the "Mask Images" button to see the masked images.
    """)
    # Displaying an image from GitHub
    url3= r'https://github.com/imadmlf/GroundingDINO/blob/main/streamlit/3.PNG?raw=true'
    resp = requests.get(url3, stream=True).raw
    image_array_heart = np.asarray(bytearray(resp.read()), dtype="uint8")
    image_heart = cv2.imdecode(image_array_heart, cv2.IMREAD_COLOR)

    st.image(cv2.cvtColor(image_heart, cv2.COLOR_BGR2RGB), caption='Mask Images')
    st.subheader("Mask and Annotate Images")
    st.write("""
        This section combines both annotation and masking functionalities. The steps are as follows:
        - Upload the images.
        - Enter the classes you want to detect.
        - Click the "Annotate Images" button to see the annotated images.
        - Click the "Mask Images" button to see the masked images.
    """)
    # Displaying an image from GitHub
    url4 = r'https://github.com/imadmlf/GroundingDINO/blob/main/streamlit/4.PNG?raw=true'
    resp = requests.get(url4, stream=True).raw
    image_array_heart = np.asarray(bytearray(resp.read()), dtype="uint8")
    image_heart = cv2.imdecode(image_array_heart, cv2.IMREAD_COLOR)

    st.image(cv2.cvtColor(image_heart, cv2.COLOR_BGR2RGB), caption='Mask and Annotate image')

    st.header("Notes and Tips")
    st.write("""
        - Ensure the images are clear and of good quality for better detection results.
        - The classes should be relevant to the content of the images for accurate annotations and masks.
    """)

    st.markdown("### Developed by SAAD & IMAD")


def app1():
    st.title('Annotated:')
    st.title('Image Segmentation Prompt ')
    st.write("This application will be for image annotating functionality.")

    # Classes input
    CLASSES = st.text_area("Enter the classes (separated by commas)", "").split(", ")

    # Grounding DINO config paths
    GROUNDING_DINO_CONFIG_PATH = '/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
    GROUNDING_DINO_CHECKPOINT_PATH = '/content/weights/groundingdino_swint_ogc.pth'

    # File uploader for multiple images
    uploaded_images = st.file_uploader("Upload images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

    # Load Grounding DINO model
    GD_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

    # List to store annotated images
    annotated_images = []

    if uploaded_images:
        for uploaded_image in uploaded_images:
            image = cv2.imdecode(np.frombuffer(uploaded_image.read(), np.uint8), cv2.IMREAD_COLOR)
            st.image(uploaded_image, caption=f"Uploaded Image: {uploaded_image.name}")  # Ajustez la hauteur selon vos besoins
            if CLASSES is not None:
                detections = GD_model.predict_with_classes(
                    image=image,
                    classes=CLASSES,
                    box_threshold=0.35,
                    text_threshold=0.25
                )
            else:
                detections = GD_model.predict(image=image, box_threshold=0.35, text_threshold=0.25)

                        # Imprimer les détections pour vérifier ce qui est retourné

            detected_classes = set(detections.class_id)
            detected_boxes = detections.xyxy
            class_id = detections.class_id

            # Vérifier si des objets ont été détectés
            if detected_boxes.shape[0] > 0:
                box_annotator = sv.BoxAnnotator()
                annotated_frame = box_annotator.annotate(
                    scene=image.copy(),
                    detections=detections,
                    skip_label=False,
                    labels=[CLASSES[i] if CLASSES else f"Class {i}" for i in class_id]
                )
                annotated_images.append((image, annotated_frame))
            else:
                st.write("Aucun objet détecté dans l'image.")
    if st.button('Annotate Images'):
        if uploaded_images:
            # Create a figure and axes
            fig, axes = plt.subplots(len(annotated_images), 2, figsize=(10, 5 * len(annotated_images)))

            # Plot each image and its annotation
            for idx, (original_image, annotated_image) in enumerate(annotated_images):
                if len(annotated_images) == 1:
                    ax1, ax2 = axes[0], axes[1]
                else:
                    ax1, ax2 = axes[idx]

                ax1.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image')
                ax1.axis('off')

                ax2.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
                ax2.set_title('Annotated Image')
                ax2.axis('off')
                # Afficher les classes détectées
                detected_classes_str = ', '.join([CLASSES[i] if CLASSES else f"Class {i}" for i in set(detections.class_id)])
                ax2.text(0, -20, f"Classes détectées : {detected_classes_str}", color='white', fontsize=12, bbox=dict(facecolor='black', alpha=0.8))


            st.pyplot(fig)
        else:
            st.write("No images have been uploaded and annotated.")

    st.text("Note: This is a demo app for image annotation.")
    st.markdown("# SAAD & IMAD")

def app2():
    st.title("Masked")
    st.title(" Image Segmentation Prompt")
    st.write("This application will be for image masking functionality.")

    # Classes input
    CLASSES = st.text_area("Enter the classes (separated by commas)", "person, chair").split(", ")

    # Grounding DINO config paths
    GROUNDING_DINO_CONFIG_PATH = '/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
    GROUNDING_DINO_CHECKPOINT_PATH = '/content/weights/groundingdino_swint_ogc.pth'

    # File uploader for multiple images
    uploaded_images = st.file_uploader("Upload images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

    # Load Grounding DINO model
    GD_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    MODEL_TYPE = "vit_h"
    CHECKPOINT_PATH = '/content/weights/sam_vit_h_4b8939.pth'

    from segment_anything import sam_model_registry, SamPredictor
    sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

    mask_predictor = SamPredictor(sam)

    # Lists to store annotated images
    annotated_images = []
    masked_images = []

    if uploaded_images:
        for uploaded_image in uploaded_images:
            image = cv2.imdecode(np.frombuffer(uploaded_image.read(), np.uint8), cv2.IMREAD_COLOR)
            st.image(uploaded_image, caption=f"Uploaded Image: {uploaded_image.name}")  # Adjust the height as needed

            detections = GD_model.predict_with_classes(
                image=image,
                classes=CLASSES,
                box_threshold=0.35,
                text_threshold=0.25
            )

            detected_boxes = detections.xyxy
            class_ids = detections.class_id
            box_annotator = sv.BoxAnnotator()
            mask_annotator = sv.MaskAnnotator(color=sv.Color.blue())

            # Annotate and mask each detected box
            for box, class_id in zip(detected_boxes, class_ids):
                box = np.array(box)
                mask_predictor.set_image(image)
                masks, _, _ = mask_predictor.predict(
                    point_coords=None,
                    point_labels=None,
                    box=box,
                    multimask_output=False
                )

                detections = sv.Detections(
                    xyxy=sv.mask_to_xyxy(masks=masks),
                    mask=masks
                )
                detections = detections[detections.area == np.max(detections.area)]

                annotated_image = box_annotator.annotate(
                    scene=image.copy(),
                    detections=detections,
                    skip_label=False,
                    labels=[CLASSES[class_id]]
                )
                masked_image = mask_annotator.annotate(scene=annotated_image.copy(), detections=detections)

                # Add the annotated and masked images to the lists
                annotated_images.append((image, annotated_image))
                masked_images.append((image, masked_image))

    if st.button('Annotate Images'):
        if annotated_images:
            # Create a figure and axes
            fig, axes = plt.subplots(len(annotated_images), 2, figsize=(10, 5 * len(annotated_images)))

            # Plot each image and its annotation
            for idx, (original_image, annotated_image) in enumerate(annotated_images):
                if len(annotated_images) == 1:
                    ax1, ax2 = axes
                else:
                    ax1, ax2 = axes[idx]

                ax1.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image')
                ax1.axis('off')

                ax2.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
                ax2.set_title('Annotated Image')
                ax2.axis('off')

            st.pyplot(fig)
        else:
            st.write("No images have been uploaded and annotated.")

    if st.button('Mask Images'):
        if masked_images:
            # Create a figure and axes
            fig, axes = plt.subplots(len(masked_images), 2, figsize=(10, 5 * len(masked_images)))

            # Plot each image and its annotation
            for idx, (original_image, masked_image) in enumerate(masked_images):
                if len(masked_images) == 1:
                    ax1, ax2 = axes
                else:
                    ax1, ax2 = axes[idx]

                ax1.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image')
                ax1.axis('off')

                ax2.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
                ax2.set_title('Masked Image')
                ax2.axis('off')

            st.pyplot(fig)
        else:
            st.write("No images have been uploaded and masked.")

    st.text("Note: This is a demo app for image masking.")
    st.markdown("# SAAD & IMAD")

def app3():

    st.title('# Masked and Annotated ')
    st.title("Image Segmentation Prompt")
    st.write("This application will be for image masking and annotating  functionality.")

    # Classes input
    CLASSES = st.text_area("Enter the classes (separated by commas)", "person, chair").split(", ")

    # Grounding DINO config paths
    GROUNDING_DINO_CONFIG_PATH = '/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
    GROUNDING_DINO_CHECKPOINT_PATH = '/content/weights/groundingdino_swint_ogc.pth'

    # File uploader for multiple images
    uploaded_images = st.file_uploader("Upload images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

    # Load Grounding DINO model
    GD_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    MODEL_TYPE = "vit_h"
    CHECKPOINT_PATH = '/content/weights/sam_vit_h_4b8939.pth'

    sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
    mask_predictor = SamPredictor(sam)

    # Lists to store annotated images
    annotated_images = []
    masked_images = []

    if uploaded_images:
        for uploaded_image in uploaded_images:
            image = cv2.imdecode(np.frombuffer(uploaded_image.read(), np.uint8), cv2.IMREAD_COLOR)
            st.image(uploaded_image, caption=f"Uploaded Image: {uploaded_image.name}")

            detections = GD_model.predict_with_classes(
                image=image,
                classes=CLASSES,
                box_threshold=0.35,
                text_threshold=0.25
            )

            detected_boxes = detections.xyxy
            class_ids = detections.class_id
            box_annotator = sv.BoxAnnotator()
            mask_annotator = sv.MaskAnnotator(color=sv.Color.blue())

            for box, class_id in zip(detected_boxes, class_ids):
                box = np.array(box)
                mask_predictor.set_image(image)
                masks, _, _ = mask_predictor.predict(
                    point_coords=None,
                    point_labels=None,
                    box=box,
                    multimask_output=False
                )

                detections = sv.Detections(
                    xyxy=sv.mask_to_xyxy(masks=masks),
                    mask=masks
                )
                detections = detections[detections.area == np.max(detections.area)]

                annotated_image = box_annotator.annotate(
                    scene=image.copy(),
                    detections=detections,
                    skip_label=False,
                    labels=[CLASSES[class_id]]
                )
                masked_image = mask_annotator.annotate(scene=annotated_image.copy(), detections=detections)

                # Add the annotated and masked images to the lists
                annotated_images.append((image, annotated_image))
                masked_images.append((image, masked_image))

    if st.button('Annotate Images'):
        if annotated_images:
            # Create a figure and axes
            fig, axes = plt.subplots(len(annotated_images), 2, figsize=(10, 5 * len(annotated_images)))

            # Plot each image and its annotation
            for idx, (original_image, annotated_image) in enumerate(annotated_images):
                if len(annotated_images) == 1:
                    ax1, ax2 = axes
                else:
                    ax1, ax2 = axes[idx]

                ax1.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image')
                ax1.axis('off')

                ax2.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
                ax2.set_title('Annotated Image')
                ax2.axis('off')

            st.pyplot(fig)
        else:
            st.write("No images have been uploaded and annotated.")

    if st.button('Mask Images'):
        if masked_images:
            # Create a figure and axes
            fig, axes = plt.subplots(len(masked_images), 2, figsize=(10, 5 * len(masked_images)))

            # Plot each image and its annotation
            for idx, (original_image, masked_image) in enumerate(masked_images):
                if len(masked_images) == 1:
                    ax1, ax2 = axes
                else:
                    ax1, ax2 = axes[idx]

                ax1.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image')
                ax1.axis('off')

                ax2.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
                ax2.set_title('Masked Image')
                ax2.axis('off')

            st.pyplot(fig)
        else:
            st.write("No images have been uploaded and masked.")

    st.text("Note: This is a demo app for image annotation.")
    st.markdown("# SAAD & IMAD")


PAGES = {
    "Home": app0,
    "Annotate Images": app1,
    "Mask Images": app2,
    "Mask and Annotate Images": app3
}

st.sidebar.title("Navigation")
selection = st.sidebar.radio("Go to", list(PAGES.keys()))

page = PAGES[selection]
page()


Overwriting App.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com
# the code of streamlit app

34.87.136.241


# url to run the app
![Image Description](https://github.com/imadmlf/GroundingDINO/blob/main/streamlit/Capture.PNG?raw=true)


In [ ]:
#url to run
! streamlit run App.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.136.241:8501

npx: installed 22 in 2.63s
your url is: https://swift-states-ask.loca.lt
/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
final text_encoder_type: bert-base-uncased
final text_encoder_type: bert-base-uncased
final text_encoder_type: bert-base-uncased
  Stopping...
^C


In [ ]:
# just annotation

%%writefile app.py

import streamlit as st
import cv2
import numpy as np
from groundingdino.util.inference import Model
import supervision as sv
import torch
import matplotlib.pyplot as plt

st.title('# Image segmentation prompt (annotation & mask) App')

# Classes input
CLASSES = st.text_area("Enter the classes (separated by commas)", "person, chair").split(", ")

# Grounding DINO config paths
GROUNDING_DINO_CONFIG_PATH = '/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
GROUNDING_DINO_CHECKPOINT_PATH = '/content/weights/groundingdino_swint_ogc.pth'

# File uploader for multiple images
uploaded_images = st.file_uploader("Upload images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

# Load Grounding DINO model
GD_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

# List to store annotated images
annotated_images = []

if uploaded_images:
    for uploaded_image in uploaded_images:
        image = cv2.imdecode(np.frombuffer(uploaded_image.read(), np.uint8), cv2.IMREAD_COLOR)
        st.image(uploaded_image, caption=f"Uploaded Image: {uploaded_image.name}")  # Ajustez la hauteur selon vos besoins


        detections = GD_model.predict_with_classes(
            image=image,
            classes=CLASSES,
            box_threshold=0.35,
            text_threshold=0.25
        )

        detected_classes = set(detections.class_id)
        detected_boxes = detections.xyxy
        class_id = detections.class_id
        box_annotator = sv.BoxAnnotator()
        annotated_frame = box_annotator.annotate(
            scene=image.copy(),
            detections=detections,
            skip_label=False,
            labels=[CLASSES[i] for i in class_id]
        )

        # Add the annotated image to the list
        annotated_images.append((image, annotated_frame))

if st.button('Annotate Images'):
    if annotated_images:
        # Create a figure and axes
        fig, axes = plt.subplots(len(annotated_images), 2, figsize=(10, 5 * len(annotated_images)))

        # Plot each image and its annotation
        for idx, (original_image, annotated_image) in enumerate(annotated_images):
            if len(annotated_images) == 1:
                ax1, ax2 = axes[0], axes[1]
            else:
                ax1, ax2 = axes[idx]

            ax1.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
            ax1.set_title('Original Image')
            ax1.axis('off')

            ax2.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
            ax2.set_title('Annotated Image')
            ax2.axis('off')

        st.pyplot(fig)
    else:
        st.write("No images have been uploaded and annotated.")

st.text("Note: This is a demo app for image annotation.")
st.markdown("# SAAD & IMAD")

Writing app.py
